In [22]:
import torch
import numpy as np
import pandas as pd
from torch.autograd import Variable

In [23]:
df=pd.read_csv("C:\\Users\\shrey\\OneDrive\\Documents\\.PES\\PIL\\fin-dataset_2.csv")
df
df.drop(['node_code'],axis=1)

,Unnamed: 0,node_type,x(m),y(m),primary_channel,min_channel_allowed,max_channel_allowed,RSSI,SINR,throughput,average_airtime,average_interference,wlan_code_index
0,0,0,10.0000,10.0000,4,4,5,-58.226667,29.62,111.77,95.745,-107.399091,0
1,1,1,0.0713,10.8079,4,4,5,-65.370000,29.36,5.79,95.745,0.000000,0
2,2,1,1.9627,4.1427,4,4,5,-65.350000,29.48,6.11,95.745,0.000000,0
3,3,1,13.7849,16.7538,4,4,5,-61.410000,31.97,6.91,95.745,0.000000,0
4,4,1,6.7112,1.7487,4,4,5,-63.520000,31.26,9.99,95.745,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87904,87904,1,22.4444,46.9925,0,0,7,-68.170000,11.06,3.30,79.220,0.000000,9
87905,87905,1,32.0104,51.2173,0,0,7,-52.610000,32.17,43.24,79.220,0.000000,9
87906,87906,1,34.1608,48.5987,0,0,7,-59.770000,24.86,37.02,79.220,0.000000,9
87907,87907,1,31.8977,54.3313,0,0,7,-60.700000,23.68,36.86,79.220,0.000000,9


In [24]:
from sklearn.model_selection import train_test_split
y=df['throughput'].values #throughput 


x=df[[  'x(m)', 'y(m)',
       'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'RSSI',
       'SINR', 'average_airtime', 'average_interference',
       'wlan_code_index']].values
       

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)
print("PyTorch Version: ", torch.__version__)
       

PyTorch Version:  1.12.0+cu113


In [25]:
from torch.utils.data.dataloader import default_collate
# default collate is to allow me to send the batches to the gpu after dataloader since 
#doesnt have an attribute called to_device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#X_train.to_numpy()
X_train = torch.from_numpy(X_train).float()
X_train
y_train = torch.squeeze(torch.from_numpy(y_train).float())
X_test = torch.from_numpy(X_test).float()
y_test = torch.squeeze(torch.from_numpy(y_test).float())
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
y_test
batch_size=512

train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True, collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x))) 


torch.Size([70327, 10]) torch.Size([70327])
torch.Size([17582, 10]) torch.Size([17582])


In [26]:
#writing the appropriate NN
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define the neural network
class Net(nn.Module):
    def __init__(self,n_features):
        super(Net,self).__init__()
        self.fc1=nn.Linear(n_features,384)
        self.fc2=nn.Linear(384,384)
        self.fc3=nn.Linear(384,384)
        self.fc4=nn.Linear(384,384)
        self.fc5=nn.Linear(384,384)
        self.fc6=nn.Linear(384,1)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.relu(self.fc4(x))
        x=F.relu(self.fc5(x))
        x=self.fc6(x)
        return x
net = Net(X_train.shape[1]) #instantiate the NN
err_fn=nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)



In [27]:
# running on cuda for better performance

#train_loader=train_loader.to(device)
#test_loader=test_loader.to(device)
net=net.to(device)
err_fn=err_fn.to(device)
#optimizer=optimizer.to(device)

In [28]:
num_epochs=1000
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
size = len(train_loader.dataset)
for epoch in range(num_epochs):
     for batch, (X, y) in enumerate(train_loader):
          pred=net(X)
          loss=err_fn(pred,y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_list.append(loss.item())
          iteration_list.append(count)
          if batch % 100 == 0:
               loss, current = loss.item(), batch * len(X)
              # tstsize = len(test_loader.dataset)
              # num_batches = len(test_loader)
              # test_loss, correct = 0, 0
              # for X, y in test_loader:
            #
              #      pred = net(X)
              #      test_loss += err_fn(pred, y).item()
              #      for i in range(len(y)):
              #           correct += (((abs(pred[i] - y[i])/y[i]) * 100) < 10).type(torch.float).item()
              # test_loss /= num_batches
              # correct /= tstsize          
            
               
               print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
               #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


c:\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 967.793091  [    0/70327]
loss: 471.563293  [51200/70327]


c:\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([183])) that is different to the input size (torch.Size([183, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 1045.117432  [    0/70327]
loss: 740.569946  [51200/70327]
loss: 841.907166  [    0/70327]
loss: 959.958008  [51200/70327]
loss: 793.673035  [    0/70327]
loss: 822.802490  [51200/70327]
loss: 781.035767  [    0/70327]
loss: 874.222046  [51200/70327]
loss: 561.860718  [    0/70327]
loss: 789.432495  [51200/70327]
loss: 564.615967  [    0/70327]
loss: 1571.572632  [51200/70327]
loss: 563.163452  [    0/70327]
loss: 856.534180  [51200/70327]
loss: 1089.685059  [    0/70327]
loss: 787.177795  [51200/70327]
loss: 986.074951  [    0/70327]
loss: 995.072266  [51200/70327]
loss: 1496.528564  [    0/70327]
loss: 357.177612  [51200/70327]
loss: 771.809326  [    0/70327]
loss: 598.800781  [51200/70327]
loss: 861.836304  [    0/70327]
loss: 433.789001  [51200/70327]
loss: 1144.667847  [    0/70327]
loss: 777.454956  [51200/70327]
loss: 647.253662  [    0/70327]
loss: 973.617188  [51200/70327]
loss: 1002.931396  [    0/70327]
loss: 769.452637  [51200/70327]
loss: 745.326477  [    0/70327]
lo

KeyboardInterrupt: 

In [ ]:
import sklearn.metrics as skm
tstsize = len(test_loader.dataset)
num_batches = len(test_loader)
test_loss, correct = 0, 0
for X, y in test_loader:

     pred = net(X)
     test_loss += err_fn(pred, y).item()
     for i in range(len(y)):
          correct += (((abs(pred[i] - y[i])/y[i]) * 100) < 10).type(torch.float).item()
test_loss /= num_batches
correct /= tstsize          
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
               